In [ ]:
import pandas as pd
import numpy as np
import re
import string
from bs4 import BeautifulSoup
from ast import literal_eval
import heapq
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
import tensorflow as tf
tf.version.VERSION

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

In [ ]:
CF  = pd.read_csv('CareersFutureCleaned.csv')

In [ ]:
CF

In [ ]:
CF['clean_jd'] = CF.apply(lambda x: str(x['Job Description']), axis=1)
CF['clean_jd'] = CF['clean_jd'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))
#lowercase all characters
CF['clean_jd'] = CF["clean_jd"].fillna('').apply(lambda x: x.lower())

In [ ]:
SFW  = pd.read_csv('sf_skills_concat.csv')

In [ ]:
SFW

In [ ]:
SFW.isna().sum()

In [ ]:
SFW.dropna(inplace=True)

In [ ]:
SFW[SFW['Job Role'].str.contains("Senior Product Manager")]['Skill Title'].unique()

In [ ]:
SFW[SFW['Job Role'].str.contains("Product Manager")]['Skill Title'].unique()

In [ ]:
SFW_grp_sector = SFW.groupby(['Job Role', 'Sector'])['Skill description'].apply(lambda x: ','.join(x)).reset_index()

In [ ]:
SFW_grp_sector

In [ ]:
def get_closest_3_sfw_job(cf_jd, cf_job_title):
    cf_jd_nlp = nlp(cf_jd)
    sim_scores = []
    sim_sfw_jd = []
    sim_sfw_job_roles = []
    
    for i in range(0, len(SFW_grp_sector)):
        sfw_jd_nlp = nlp(SFW_grp_sector['Skill description'][i])
        sim_scores.append(sfw_jd_nlp.similarity(cf_jd_nlp))
        sim_sfw_jd.append(SFW_grp_sector['Skill description'][i])
        sim_sfw_job_roles.append(SFW_grp_sector['Job Role'][i])
        
    top_3_scores = heapq.nlargest(3, range(len(sim_scores)), key=fr.__getitem__)
    top_3_sfw_jobs = heapq.nlargest(3, range(len(sim_sfw_job_roles)), key=fr.__getitem__)
    top_3_sfw_jd = heapq.nlargest(3, range(len(sim_sfw_jd)), key=fr.__getitem__)
    
    return top_3_scores, top_3_sfw_jobs, top_3_sfw_jd

In [ ]:
get_closest_3_sfw_job(CF['clean_jd'][0], CF['Job Title'][0])

In [ ]:
x = SFW['Job Role'].unique()

In [ ]:
unique_job_roles = pd.DataFrame(x, columns = ['unique job roles'])

In [ ]:
unique_job_roles

In [ ]:
x = SFW[SFW['Job Role'].str.contains("Production Supervisor")]

In [ ]:
x

In [ ]:
y = SFW[SFW['Job Role'].str.contains("Senior Maintenance Technician")]

In [ ]:
y

In [ ]:
a = nlp(x['Skill description'][40524])
b = nlp(y['Skill description'][4162])
a.similarity(b)

In [ ]:
SFW[SFW['Skill Title'].str.contains("Workplace Safety and Health System Management")]

In [ ]:
SFW['Skill description'][35090]

In [ ]:
SFW['Skill description'][35091]

In [ ]:
x = SFW[SFW['Job Role'].str.contains("Airport Emergency Officer")][:15]

In [ ]:
x

In [ ]:
SFW[SFW['Skill Title'].str.contains("Change Management")]

In [ ]:
len(x)

In [ ]:
y = SFW[SFW['Job Role'].str.contains("Airport Emergency Officer")][15:]

In [ ]:
y

In [ ]:
len(y)
# skills of Airport Emergency Officer is a subset of Airport Emergency Officer specialist so can grp these 2 jobs
# tgt by removing Airport Emergency Officer specialist and adding its additional skills to Airport Emergency Officer
# need to find other groups of jobs (e.g. smth with lead xxx and xxx or senior xxx and xxx) to do same to reduce 
# 1000+ unique jobs to fewer unique jobs - need to see if will affect skills library part
# use fuzzy matching -> for fuzzy matched job role pairs check if one job roles skills are a subset of other job role
# -> if so keep the superset job role and remove the subset job role

In [ ]:
df_selected = SFW[['Skill description', 'Sector', 'Job Role']]

In [ ]:
df_selected.dropna(inplace=True)

In [ ]:
df_selected

In [ ]:
df_selected.loc[13499]

In [ ]:
df_selected['Skill description'] = df_selected['Skill description'].astype(str)

In [ ]:
df_selected['Sector'] = df_selected['Sector'].astype(str)

In [ ]:
len(df_selected['Job Role'].unique())

In [ ]:
pd.get_dummies(df_selected['Job Role'])

In [ ]:
df_selected['Job Role'].unique()

In [ ]:
df_selected = pd.concat([df_selected, pd.get_dummies(df_selected['Job Role'])], axis=1)

In [ ]:
df_selected

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df_selected['Skill description'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
X = tokenizer.texts_to_sequences(df_selected['Skill description'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

In [ ]:
Y = pd.get_dummies(df_selected['Job Role']).values
print('Shape of label tensor:', Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1424, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
epochs = 10
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
job_cols = pd.get_dummies(df_selected['Job Role']).columns

In [ ]:
pred = model.predict(X_test)

In [ ]:
pred.shape

In [ ]:
len(pred[0])

In [ ]:
pred[0]

In [ ]:
X_test_texts = tokenizer.sequences_to_texts(X_test)

In [ ]:
pred_sf_job_role = []
actual_sf_job_role = []
for i in range(0, len(pred)):
    ind = np.argmax(pred[i])
    pred_sf_job_role.append(job_cols[np.argmax(pred[i])])
    actual_sf_job_role.append(job_cols[np.argmax(Y_test[i])])

In [ ]:
df = {'SF Skill Descriptions': X_test_texts, 'Actual SF Job Roles':actual_sf_job_role, 'Predicted SF Job Roles':pred_sf_job_role}

In [ ]:
SF_test_results = pd.DataFrame(df)

In [ ]:
SF_test_results.head(50)

In [ ]:
a = [CF['clean_jd'][0]]

In [ ]:
CF['Job Title'][0]

In [ ]:
seq1 = tokenizer.texts_to_sequences(a)
padded1 = pad_sequences(seq1, maxlen=MAX_SEQUENCE_LENGTH)
pred_a = model.predict(padded1)

In [ ]:
pred_a[0]

In [ ]:
np.argmax(pred_a)

In [ ]:
job_cols[np.argmax(pred_a[0])]

In [ ]:
heapq.nlargest(3, range(len(pred_a[0])), key=pred_a[0].__getitem__)

In [ ]:
job_cols[596]

In [ ]:
all_pred_cf = []
actual_cf_job_role = []
for i in range(50):
    new_cf_jd = [CF['clean_jd'][i]]
    actual_cf_job_role.append(CF['Job Title'][i])
    seq = tokenizer.texts_to_sequences(new_cf_jd)
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    pred_cf = model.predict(padded)
    all_pred_cf.append(pred_cf)

In [ ]:
pred_sf_job_role = []
for i in range(0, len(all_pred_cf)):
    ind = np.argmax(all_pred_cf[i])
    pred_sf_job_role.append(job_cols[np.argmax(all_pred_cf[i])])

In [ ]:
df_cf_pred = {'Actual CF Job Title':actual_cf_job_role, 'Predicted SF Job Role':pred_sf_job_role}

In [ ]:
CF_test_results = pd.DataFrame(df_cf_pred)

In [ ]:
CF_test_results

# SGD classifier and Random forest

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [ ]:
jobs = df_selected['Job Role'].unique()
train, test = train_test_split(df_selected, random_state=42, test_size=0.33, shuffle=True)
X_train = train['Skill description']
y_train = train['Job Role']
X_test = test['Skill description']
y_test = train['Job Role']
print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
train['QA Engineer']

In [ ]:
(test['Technical Officer']==1).sum()

In [ ]:
# Define a pipeline combining a text feature extractor with multi label classifier
NB_pipeline = Pipeline([
                #('vect', CountVectorizer(lowercase=False)),
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
for i in range(0, len(jobs)):
    print(i)
    print('... Processing {}'.format(jobs[i]))
    # train the model using X_dtm & y
    NB_pipeline.fit(X_train, train[jobs[i]]) #each time fit model with diff job 
    #does 1424 individual classifications taking y_train as each job role each time
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_test) #predict for all test rows and job i
    print('Test accuracy is {}'.format(accuracy_score(test[jobs[i]], prediction)))
    
#20mins for 100 job roles, 4-5hrs for 1424 job roles

In [ ]:
len(prediction)

In [ ]:
(prediction==1).sum()

In [ ]:
sum(prediction)

In [ ]:
# Define a pipeline combining a text feature extractor with multi label classifier
RF_pipeline = Pipeline([
                #('vect', CountVectorizer(lowercase=False)),
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', RandomForestClassifier(max_depth=2, random_state=0)),
            ])
predictions = []
for i in range(0, len(jobs)):
    print(i)
    print('... Processing {}'.format(jobs[i]))
    # train the model using X_dtm & y
    RF_pipeline.fit(X_train, train[jobs[i]]) #each time fit model with diff job 
    #does 1424 individual classifications taking y_train as each job role each time
    # compute the testing accuracy
    pred = RF_pipeline.predict(X_test) #predict for all test rows and job i
    predictions.append(pred)
    print('Test accuracy is {}'.format(accuracy_score(test[jobs[i]], pred)))
    
#20mins for 100 job roles, 4-5hrs for 1424 job roles

In [ ]:
predictions

In [ ]:
predictions[0].sum()

In [ ]:
predictions[1].sum()

In [ ]:
predictions[2].sum()

In [ ]:
predictions[3].sum()

In [ ]:
X = df_selected[['Skill description']]
y = df_selected[['Job Role']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([#('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)


y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)


y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=ind))
RandomForestClassifier()

In [ ]:
CF['Industry_cleaned'] = CF['Industry'].apply(lambda x : eval(x))

In [ ]:
selection = ['Design']
selected = CF.Industry_cleaned.apply(lambda x: any(item for item in selection if item in x))
CF_selected = CF[selected]

In [ ]:
CF_selected.reset_index(inplace=True)

In [ ]:
CF_selected

In [ ]:
new_jd = CF['Job Description'][1]
seq = tokenizer.texts_to_sequences(new_jd)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
# labels = SFW['Job Role'].unique()
# print(pred, labels[np.argmax(pred)])

In [ ]:
np.argmax(pred)

# NEW

In [ ]:
sfw_new = pd.read_csv('sf_model_final.csv')

In [ ]:
sfw_new

In [ ]:
sfw_new.isna().sum()

In [ ]:
sfw_new.dropna(inplace=True)

In [ ]:
df_selected = sfw_new[sfw_new['Sector'] == "Infocomm Technology"]

In [ ]:
df_selected_new = df_selected[['Skill description', 'Sector', 'Job_Role_Replaced']]

In [ ]:
df_selected_new

In [ ]:
len(df_selected_new['Job_Role_Replaced'].unique())

In [ ]:
df_selected_new['Job_Role_Replaced'].unique()

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df_selected_new['Skill description'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
X = tokenizer.texts_to_sequences(df_selected_new['Skill description'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

In [ ]:
Y = pd.get_dummies(df_selected_new['Job_Role_Replaced']).values
print('Shape of label tensor:', Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(87, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
jobs = pd.get_dummies(df_selected_new['Job_Role_Replaced']).columns

In [ ]:
pred = model.predict(X_test)

In [ ]:
pred.shape

In [ ]:
X_test_texts = tokenizer.sequences_to_texts(X_test)

In [ ]:
pred_sf_job_role = []
actual_sf_job_role = []
for i in range(0, len(pred)):
    ind = np.argmax(pred[i])
    pred_sf_job_role.append(jobs[np.argmax(pred[i])])
    actual_sf_job_role.append(jobs[np.argmax(Y_test[i])])

In [ ]:
df = {'SF Skill Descriptions': X_test_texts, 'Actual SF Job Roles':actual_sf_job_role, 'Predicted SF Job Roles':pred_sf_job_role}
SF_test_results = pd.DataFrame(df)

In [ ]:
SF_test_results.head(30)

In [ ]:
df_selected_new['Job_Role_Replaced'].value_counts()